In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import create_auction_app, setup_auction_app, place_bid, close_auction
from store.operations import StoringPool
from utils import get_algod_client, wait_for_confirmation, get_balances, get_last_block_timestamp
from account import Account

In [2]:
dotenv.load_dotenv('.env')
print(os.environ.get('ALGOD_URL'))

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

http://localhost:4001


In [3]:
def optin_to_asset(asset_id, sender: Account):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    wait_for_confirmation(client, signed_txn.get_txid())

In [4]:
creator = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
seller = Account.from_mnemonic(os.environ.get("SELLER_MN"))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [5]:
def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        decimals=decimals,
        asset_name=asset_name,
        unit_name=unit_name,
        default_frozen=False,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")
print(f"The token id is: {asset_id}")

Waiting for confirmation...
Transaction ELUFDFFO2KKDOVMZB5M5E35EKUZQAG7TJ65YQNIGAPC22ZWZSG2Q confirmed in round 2055.
The token id is: 5


In [6]:
start_time = int(time()) + 10
end_time = start_time + 10
reserve = 1_000_000  # 1 Algo
token_amount = 1_000_000
increment = 100_000  # 0.1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [8]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)
storeApp.create_app()
storeAppID = storeApp.app_id
print(f"Store App ID: {storeAppID}")



print(f"seller address: {seller.get_address()}")
appID = create_auction_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment,
    store_app_address=get_application_address(storeAppID)
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

AlgodHTTPError: TransactionPool.Remember: transaction DKCRXPE6LYMTV3MMRW4XH4TPEYYNASHSOXEIZ2ZE2XTJIQ3GHZNA: overspend (account LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I, data {_struct:{} Status:Offline MicroAlgos:{Raw:0} RewardsBase:0 RewardedMicroAlgos:{Raw:0} VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0 AssetParams:map[] Assets:map[] AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ AppLocalStates:map[] AppParams:map[] TotalAppSchema:{_struct:{} NumUint:0 NumByteSlice:0} TotalExtraAppPages:0}, tried to spend {1000})

Alice is setting up and funding token auction...

In [ ]:
setup_auction_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=1
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

In [ ]:
bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))

print(f"Bidder Address: {bidder.get_address()}")

In [ ]:
_, last_round_time = get_last_block_timestamp(client)
if last_round_time < start_time + 5:
    sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
bidAmount = reserve
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

In [ ]:
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

In [ ]:
optin_to_asset(asset_id, bidder)

Alice is closing out the auction....

In [ ]:
_, lastRoundTime = get_last_block_timestamp(client)
if lastRoundTime < end_time + 5:
    sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, seller)

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0